In [1]:
link = 'https://cs.hse.ru/persons'

In [2]:
from bs4 import BeautifulSoup
import requests
import re

In [18]:
soup = BeautifulSoup(requests.get(link).text, 'lxml')
persons = soup.find_all('a', {"class":"fa-person__name"})
persons_links = ['http://' + a.get('href').lstrip('//') for a in persons]
first_ten = persons_links[:10]

In [19]:
first_ten

['http://www.hse.ru/staff/arjantsev',
 'http://www.hse.ru/staff/voznesenskaya',
 'http://www.hse.ru/staff/obiedkov',
 'http://www.hse.ru/staff/igergart',
 'http://www.hse.ru/staff/samonenko',
 'http://www.hse.ru/org/persons/113170202',
 'http://www.hse.ru/staff/abashidze',
 'http://www.hse.ru/org/persons/14279019',
 'http://www.hse.ru/org/persons/150045920',
 'http://www.hse.ru/org/persons/348330900']

In [52]:
for link in first_ten:
    page = requests.get(link).text
    if 'Резюме' in page:
        soup_link = BeautifulSoup(page, 'lxml')
        cv_link = 'https://www.hse.ru' + soup_link.find_all('a', text='Резюме')[0].get('href')
        name = re.findall(r'Сотрудники - ([\w\s]+\b)', soup_link.find_all('title')[0].get_text())[0]
        doc = requests.get(cv_link)
        file_type = doc.headers['Content-Type'].split('/')[1]
        fh = open(f'{name}.{file_type}', 'wb')
        fh.write(doc.content)
        fh.close()
    

In [59]:
lang_dict = {}
for link in first_ten:
    page = requests.get(link).text
    if 'Резюме' in page:
        soup_link = BeautifulSoup(page, 'lxml')
        name = re.findall(r'Сотрудники - ([\w\s]+\b)', soup_link.find_all('title')[0].get_text())[0]
        lang = soup_link.find_all('dl', {'class':"main-list large main-list-language-knowledge-level"})[0]
        lang_dict[name] = [l.get_text() for l in lang.find_all('dd')]
        
print(lang_dict)

{'Аржанцев Иван Владимирович': ['английский', 'украинский', 'немецкий', 'французский'], 'Объедков Сергей Александрович': ['английский', 'французский'], 'Самоненко Илья Юрьевич': ['английский'], 'Абашидзе Хатуна Датоевна': ['английский', 'грузинский', 'немецкий', 'русский', 'турецкий'], 'Масютин Алексей Александрович': ['английский', 'французский'], 'Абовьян Наталья Михайловна': ['английский', 'французский (начальные знания)', 'испанский (начальные знания)']}
